In [1]:
# Mohammadmilad Sayyad____801419978

# Problem 2(a): Linear regression on Housing (area, bedrooms, bathrooms, stories, parking)
# - 80/20 split (fixed seed)
# - z-score standardization using TRAIN stats
# - Linear regression (5 weights + bias) optimized with SGD
# - Prints params on standardized features AND equivalent params on original feature scale

import numpy as np
import pandas as pd
import torch

# --------- Load data ---------
# If using Colab, put Housing.csv in your /content/ working dir or mount Drive and set the path.
CSV_PATH = "Housing.csv"  # change if needed
df = pd.read_csv(CSV_PATH)

features = ["area","bedrooms","bathrooms","stories","parking"]
target   = "price"

X = df[features].values.astype(np.float32)
y = df[target].values.astype(np.float32).reshape(-1,1)

# --------- Split 80/20 with fixed seed for reproducibility ---------
n = len(df)
idx = np.arange(n)
rng = np.random.default_rng(0)
rng.shuffle(idx)
train_size = int(0.8 * n)
train_idx, val_idx = idx[:train_size], idx[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_val,   y_val   = X[val_idx],   y[val_idx]

# --------- Standardize inputs (train mean/std only) ---------
mu = X_train.mean(axis=0)
sigma = X_train.std(axis=0)
sigma[sigma == 0] = 1.0  # safety

X_train_n = (X_train - mu) / sigma
X_val_n   = (X_val   - mu) / sigma

# Torch tensors
Xt = torch.tensor(X_train_n)
yt = torch.tensor(y_train)
Xv = torch.tensor(X_val_n)
yv = torch.tensor(y_val)

# --------- Linear model: y = XW + b ---------
torch.manual_seed(0)
W = torch.zeros((Xt.shape[1], 1), requires_grad=True)  # 5x1
b = torch.zeros(1, requires_grad=True)

def forward(X):
    return X @ W + b

mse = torch.nn.MSELoss()
opt = torch.optim.SGD([W, b], lr=1e-2)  # lr consistent with P1; P2(b) will sweep LRs/epochs

EPOCHS = 5000  # you can keep 5000 so P2(b) can reuse; logs omitted here
for _ in range(EPOCHS):
    yhat = forward(Xt)
    loss = mse(yhat, yt)
    opt.zero_grad(); loss.backward(); opt.step()

# --------- Metrics ---------
with torch.no_grad():
    train_mse = mse(forward(Xt), yt).item()
    val_mse   = mse(forward(Xv), yv).item()

    # R^2 helper
    def r2_score(y_true, y_pred):
        y_true = y_true.numpy(); y_pred = y_pred.numpy()
        ss_res = ((y_true - y_pred)**2).sum()
        ss_tot = ((y_true - y_true.mean())**2).sum()
        return 1.0 - ss_res/ss_tot

    train_r2 = r2_score(yt, forward(Xt))
    val_r2   = r2_score(yv, forward(Xv))

# --------- Parameters on standardized features ---------
W_std = W.detach().numpy().flatten()
b_std = float(b.detach().item())

# --------- Equivalent parameters on ORIGINAL feature scale ---------
# If the model is y = W_std^T * ((x - mu)/sigma) + b_std
# then y = (W_std/sigma)^T * x + (b_std - sum(W_std * mu/sigma))
W_raw = W_std / sigma
b_raw = b_std - float((W_std * mu / sigma).sum())

print("\n=== TRAIN/VAL METRICS ===")
print(f"Train MSE: {train_mse:,.2f}   |  Train R^2: {train_r2:.3f}")
print(f"Val   MSE: {val_mse:,.2f}   |  Val   R^2: {val_r2:.3f}")

print("\n=== PARAMETERS (Standardized Inputs) ===")
for name, w in zip(features, W_std):
    print(f"W[{name:10s}] = {w: .6f}")
print(f"B = {b_std: .6f}")

print("\n=== PARAMETERS (Original Feature Scale) ===")
for name, w in zip(features, W_raw):
    print(f"W_raw[{name:10s}] = {w:,.3f}")
print(f"B_raw = {b_raw:,.3f}")



=== TRAIN/VAL METRICS ===
Train MSE: 1,470,189,797,376.00   |  Train R^2: 0.584
Val   MSE: 1,792,476,971,008.00   |  Val   R^2: 0.462

=== PARAMETERS (Standardized Inputs) ===
W[area      ] =  712531.937500
W[bedrooms  ] =  81994.570312
W[bathrooms ] =  597510.625000
W[stories   ] =  504446.093750
W[parking   ] =  295760.687500
B =  4777254.000000

=== PARAMETERS (Original Feature Scale) ===
W_raw[area      ] = 351.522
W_raw[bedrooms  ] = 114,478.633
W_raw[bathrooms ] = 1,194,126.250
W_raw[stories   ] = 565,091.500
W_raw[parking   ] = 348,539.625
B_raw = -168,828.500
